In [21]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score, precision_recall_curve, f1_score
from sklearn.utils import shuffle

In [22]:
bank = pd.read_csv('Churn.csv')
bank.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [23]:
bank = bank.drop('Surname', axis=1)
bank = pd.get_dummies(bank, columns=['Geography', 'Gender'], drop_first=True)
print(bank)

      RowNumber  CustomerId  CreditScore  Age  Tenure    Balance  \
0             1    15634602          619   42     2.0       0.00   
1             2    15647311          608   41     1.0   83807.86   
2             3    15619304          502   42     8.0  159660.80   
3             4    15701354          699   39     1.0       0.00   
4             5    15737888          850   43     2.0  125510.82   
...         ...         ...          ...  ...     ...        ...   
9995       9996    15606229          771   39     5.0       0.00   
9996       9997    15569892          516   35    10.0   57369.61   
9997       9998    15584532          709   36     7.0       0.00   
9998       9999    15682355          772   42     3.0   75075.31   
9999      10000    15628319          792   28     NaN  130142.79   

      NumOfProducts  HasCrCard  IsActiveMember  EstimatedSalary  Exited  \
0                 1          1               1        101348.88       1   
1                 1          0   

In [24]:
target = bank['Exited']
features = bank.drop('Exited', axis=1)
print(target.value_counts(normalize=True))
train_target, valid_target, train_features, valid_features = train_test_split(target, features, test_size=0.25, random_state=12345)

Exited
0    0.7963
1    0.2037
Name: proportion, dtype: float64


In [31]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

features_valid_upsampled, target_valid_upsampled = upsample(valid_features, valid_target, 4)
features_train_upsampled, target_train_upsampled = upsample(train_features, train_target, 4)
print(target_valid_upsampled.value_counts(normalize=True))

Exited
1    0.521315
0    0.478685
Name: proportion, dtype: float64
